# Import non-AiiDA database

Press the "Edit App" button to see more details

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
from aiida.orm import load_node, Group

from aiida.orm.data.cif import CifData
from aiida.orm.data.parameter import ParameterData
from aiida.work import workfunction

import re
import os 
import csv
import ase.io
import numpy as np
import pandas as pd

In [ ]:
folder_db = '/project/cofs'
structure_folder = folder_db + '/structures/'
properties_csv = folder_db + '/properties.csv'
group_name = 'parameters' # all ParameterDatas will be put in this group
                          # use `verdi export create -g <group_name>` to export everything

In [ ]:
data = None
def parse_csv(path):
    global data
    data = pd.read_csv(path, low_memory=False, verbose=1, skipinitialspace=True)
    print("Read {} data rows from .csv file".format(len(data)))

In [ ]:
prms_dict = {}
@workfunction
def link_parameters(cif):
    return ParameterData(dict=prms_dict)

In [ ]:
def store_structures_and_properties(start_from=None, end_at=None):
    global prms_dict, data
    labels = data.keys()
        
    if start_from is None:
        start_from = 0
    if end_at is None:
        end_at = len(data)
    
    params_group, success = Group.get_or_create(name=group_name)
               
    unit_regex = re.compile('\[(.*?)\]')
    for i in range(start_from, end_at):
               
        cif = CifData(file=structure_folder + data['name'][i]+'.cif', scan_type='flex', parse_policy='lazy')
        cif.description = data['name'][i]

        prms_dict = {}
        for label in labels:
            match = re.search(unit_regex, label)
            if match:
                units = match.group(1).strip()
                label_new = re.sub(unit_regex,'',label).strip().replace(' ', '_')
                prms_dict[label_new] = data[label][i]
                prms_dict[label_new+'_units'] = units
            else:
                label_new = label.replace(' ', '_')
                prms_dict[label_new] = data[label][i]

        params = link_parameters(cif=cif)
        # add cif_uuid extra for faster querying
        params.set_extra('cif_uuid', cif.uuid)
        # add to params_group for easy export
        params_group.add_nodes(params)        
        
        
        if i % 100 == 0:
            print("Stored structure no. {}, {}".format(i, data['name'][i]))

In [ ]:
#parse_csv(properties_csv)

In [ ]:
#store_structures_and_properties(0,69840)